# Divide and Conquer [Week 8]

- Break a problem into smaller subproblems.
- Solve each subproblem independently.
- Combine solutions to solve the original problem.

Examples:

- **Merge sort**: Sorts an array by dividing and merging.
- **Quick sort**: Sorts by partitioning around a pivot.
- **Binary search**: Finds an element in a sorted array.
- **Counting inversions**: Counts out-of-order pairs in an array.
- **Closest pair of points**: Finds the closest pair among a set of points.
- **Integer multiplication**: Multiplies large integers efficiently.
- **Quick select and Fast select**: Finds the k-th smallest element.
- **Median of Medians (MoM)**: Improves pivot choice in selection algorithms.
- **Fast select using MoM**: Selects an element quickly with better pivoting.

# Count Inversions Problem

https://pdsaiitm.github.io/week-8/summary.html#counting-inversions

https://itzsyboo.medium.com/algorithms-studynote-2-divide-and-conquer-counting-inversions-9e0899bc9e26

**Problem Statement:**

- Given a list of integers.
- Count the number of inversions.
- An inversion is a pair (i, j) where i < j and A[i] > A[j].

**Solution Approach:**

1. **Divide the Array:**
   - Recursively split the array into two halves.

2. **Count Inversions:**
   - In each half, count inversions using the same method.

3. **Merge and Count:**
   - Merge two sorted halves.
   - Count inversions where elements in the left half are greater than elements in the right half.

4. **Combine Counts:**
   - Total inversions = inversions in left half + inversions in right half + inversions during merging.

**Note:**
- This approach is similar to merge sort but includes inversion counts during the merge step.

In [45]:
def merge_and_count(left, right):
  merged = []
  i = j = 0
  count = 0

  # merge the two halves
  while i < len(left) and j < len(right):
    if left[i] <= right[j]:
      merged.append(left[i])
      i += 1
    else:
      merged.append(right[j])
      count += len(left) - i
      j += 1

  # append remaining elements
  merged.extend(left[i:])
  merged.extend(right[j:])

  return merged, count


def inversion_count(arr):
  if len(arr) < 2:
    return arr, 0

  mid = len(arr) // 2
  left, count_left = inversion_count(arr[:mid])
  right, count_right = inversion_count(arr[mid:])
  merged, count_merge = merge_and_count(left, right)
  return merged, count_left + count_right + count_merge


# test
arr = [2, 4, 3, 1, 5]
sorted_arr, inversions = inversion_count(arr)
print(sorted_arr, inversions)

[1, 2, 3, 4, 5] 4


**Input:** [2, 4, 3, 1, 5]

**Process:**

1. **Split Array:**
   - Left: [2, 4]
   - Right: [3, 1, 5]

2. **Count Inversions in Left:**
   - Split further: [2] and [4]
     - No inversions in [2] and [4]
   - Combine [2] and [4] (no new inversions)

3. **Count Inversions in Right:**
   - Split further: [3] and [1, 5]
     - Inversions in [1, 5]: 0
     - Merge [3] and [1, 5] with inversions: 1 (pair: (3, 1))

4. **Merge and Count:**
   - Merge sorted halves [2, 4] and [1, 3, 5]:
     - Inversions during merge: 3 (pairs: (2, 1), (4, 1), (4, 3))

5. **Total Count:**
   - Total inversions: 0 (left) + 1 (right) + 3 (merge) = 4

**Result:**

- **Number of Inversions:** 4